# Invoke functions from module

In [5]:
import sys
sys.path.append('./modules')

In [8]:
from script_module_02a import save_property, filter_properties

In [9]:
filter_properties()

6,5828 PEPPERMILL CT,SACRAMENTO,95841,CA,3,1,1122,Condo,Wed May 21 00:00:00 EDT 2008,89921,38.662595,-121.327813

9,11150 TRINITY RIVER DR Unit 114,RANCHO CORDOVA,95670,CA,2,2,941,Condo,Wed May 21 00:00:00 EDT 2008,94905,38.621188,-121.270555

16,7340 HAMDEN PL,SACRAMENTO,95842,CA,2,2,1134,Condo,Wed May 21 00:00:00 EDT 2008,110700,38.700051,-121.351278

18,6236 LONGFORD DR Unit 1,CITRUS HEIGHTS,95621,CA,2,1,795,Condo,Wed May 21 00:00:00 EDT 2008,116250,38.679776,-121.314089

31,5201 LAGUNA OAKS DR Unit 140,ELK GROVE,95758,CA,2,2,1039,Condo,Wed May 21 00:00:00 EDT 2008,133000,38.423251,-121.444489

35,5201 LAGUNA OAKS DR Unit 162,ELK GROVE,95758,CA,2,2,1039,Condo,Wed May 21 00:00:00 EDT 2008,141000,38.423251,-121.444489

40,2442 LARKSPUR LN,SACRAMENTO,95825,CA,1,1,760,Condo,Wed May 21 00:00:00 EDT 2008,150000,38.58514,-121.403736

41,4800 WESTLAKE PKWY Unit 2109,SACRAMENTO,95835,CA,2,2,1304,Condo,Wed May 21 00:00:00 EDT 2008,152000,38.658812,-121.542345

95,4010 ALEX LN,CARMICHAEL,95608

# Pass MLRun context and run function from module

In [10]:
from mlrun import get_or_create_ctx
mlrun_ctx = get_or_create_ctx('mlrun_test')

> 2020-08-05 12:49:19,855 [info] logging run results to: http://mlrun-api:8080


In [11]:
source_file = "/User/igztraining/mlrun/data/demo.csv"
target_file = "/User/igztraining/mlrun/data/condos.csv" 

In [12]:
from module_handler_functions import save_property

In [13]:
save_property(mlrun_ctx,source_file=source_file,target_file=target_file)

file saved to /User/igztraining/mlrun/data/condos.csv


# Run local with MLRun

In [14]:
from mlrun import run_local

In [15]:
params = {"source_file" : "/User/igztraining/mlrun/data/demo.csv",
"target_file" : "/User/igztraining/mlrun/data/condos.csv" }

In [16]:
run_result = run_local(name='filter_properties',project='Iguazio-Training',params=params,handler='filter_properties',command='modules/module_handler_functions.py')

> 2020-08-05 12:51:18,674 [warning] artifact path is not defined or is local, artifacts will not be visible in the UI
> 2020-08-05 12:51:18,683 [info] starting run filter_properties uid=c3a3a02cb5114662a60658561591ce86  -> http://mlrun-api:8080
6,5828 PEPPERMILL CT,SACRAMENTO,95841,CA,3,1,1122,Condo,Wed May 21 00:00:00 EDT 2008,89921,38.662595,-121.327813

9,11150 TRINITY RIVER DR Unit 114,RANCHO CORDOVA,95670,CA,2,2,941,Condo,Wed May 21 00:00:00 EDT 2008,94905,38.621188,-121.270555

16,7340 HAMDEN PL,SACRAMENTO,95842,CA,2,2,1134,Condo,Wed May 21 00:00:00 EDT 2008,110700,38.700051,-121.351278

18,6236 LONGFORD DR Unit 1,CITRUS HEIGHTS,95621,CA,2,1,795,Condo,Wed May 21 00:00:00 EDT 2008,116250,38.679776,-121.314089

31,5201 LAGUNA OAKS DR Unit 140,ELK GROVE,95758,CA,2,2,1039,Condo,Wed May 21 00:00:00 EDT 2008,133000,38.423251,-121.444489

35,5201 LAGUNA OAKS DR Unit 162,ELK GROVE,95758,CA,2,2,1039,Condo,Wed May 21 00:00:00 EDT 2008,141000,38.423251,-121.444489

40,2442 LARKSPUR LN,SACRA

project,uid,iter,start,state,name,labels,inputs,parameters,results,artifacts
Iguazio-Training,...1591ce86,0,Aug 05 12:51:18,completed,filter_properties,v3io_user=adminkind=owner=adminhost=admin-jupyter-6b4b68d849-z6wxt,,source_file=/User/igztraining/mlrun/data/demo.csvtarget_file=/User/igztraining/mlrun/data/condos.csv,,


to track results use .show() or .logs() or in CLI: 
!mlrun get run c3a3a02cb5114662a60658561591ce86 --project Iguazio-Training , !mlrun logs c3a3a02cb5114662a60658561591ce86 --project Iguazio-Training
> 2020-08-05 12:51:18,908 [info] run executed, status=completed


## Run local with MLRun sdk hooks

In [17]:
run_result = run_local(name='save_property',artifact_path='/User/artifacts',project='Iguazio-Training',params=params,handler='save_property',command='modules/mlrun_functions.ipynb')

> 2020-08-05 12:54:55,633 [info] starting run save_property uid=8d6510bf0f134a4b884dc6b971ced08b  -> http://mlrun-api:8080
/User/igztraining/mlrun/data/demo.csv


project,uid,iter,start,state,name,labels,inputs,parameters,results,artifacts
Iguazio-Training,...71ced08b,0,Aug 05 12:54:55,completed,save_property,v3io_user=adminkind=owner=adminhost=admin-jupyter-6b4b68d849-z6wxt,,source_file=/User/igztraining/mlrun/data/demo.csvtarget_file=/User/igztraining/mlrun/data/condos.csv,properties_available=54,condo_properties


to track results use .show() or .logs() or in CLI: 
!mlrun get run 8d6510bf0f134a4b884dc6b971ced08b --project Iguazio-Training , !mlrun logs 8d6510bf0f134a4b884dc6b971ced08b --project Iguazio-Training
> 2020-08-05 12:54:55,927 [info] run executed, status=completed


# Deploy code as an MLRun function

In [18]:
from mlrun import code_to_function, mlconf,mount_v3io

In [19]:
fn = code_to_function(project='Iguazio-Training',name='save_properities', kind='job',filename='modules/mlrun_functions.ipynb')

In [20]:
fn.spec.image = "mlrun/mlrun"

In [22]:
fn.apply(mount_v3io())

In [28]:
print(fn.to_yaml())

kind: job
metadata:
  name: save-properities
  tag: ''
  project: Iguazio-Training
spec:
  command: ''
  args: []
  image: mlrun/mlrun
  volumes:
  - flexVolume:
      driver: v3io/fuse
      options:
        accessKey: 9dce1ad2-ba7f-411b-a267-a12ee278bb91
        container: users
        subPath: /admin
    name: v3io
  volume_mounts:
  - mountPath: /User
    name: v3io
  env:
  - name: V3IO_API
    value: v3io-webapi.default-tenant.svc:8081
  - name: V3IO_USERNAME
    value: admin
  - name: V3IO_ACCESS_KEY
    value: 9dce1ad2-ba7f-411b-a267-a12ee278bb91
  default_handler: ''
  entry_points:
    save_property:
      name: save_property
      doc: ''
      parameters:
      - name: context
        default: ''
      - name: source_file
        default: ''
      - name: target_file
        default: ''
      outputs:
      - default: ''
      lineno: 7
    filter_properties:
      name: filter_properties
      doc: ''
      parameters:
      - name: context
        default: ''
      - nam

### Run interactive

In [25]:
sp_run = fn.run(name='run_withcode',artifact_path="/User/artifacts",handler='save_property',params=params,project='Iguazio-Training')

> 2020-08-05 13:09:18,940 [info] starting run run_withcode uid=03494c3da2414db5ad6a04c30f306cd5  -> http://mlrun-api:8080
> 2020-08-05 13:09:19,087 [info] Job is running in the background, pod: run-withcode-vbl62
/User/igztraining/mlrun/data/demo.csv
> 2020-08-05 13:09:23,615 [info] run executed, status=completed
final state: succeeded


project,uid,iter,start,state,name,labels,inputs,parameters,results,artifacts
Iguazio-Training,...0f306cd5,0,Aug 05 13:09:23,completed,run_withcode,v3io_user=adminkind=jobowner=adminhost=run-withcode-vbl62,,source_file=/User/igztraining/mlrun/data/demo.csvtarget_file=/User/igztraining/mlrun/data/condos.csv,properties_available=54,condo_properties


to track results use .show() or .logs() or in CLI: 
!mlrun get run 03494c3da2414db5ad6a04c30f306cd5 --project Iguazio-Training , !mlrun logs 03494c3da2414db5ad6a04c30f306cd5 --project Iguazio-Training
> 2020-08-05 13:09:25,241 [info] run executed, status=completed


In [26]:
sp_run.outputs

{'properties_available': 54,
 'condo_properties': 'store://Iguazio-Training/run_withcode_condo_properties#03494c3da2414db5ad6a04c30f306cd5'}

### Run and forget

In [27]:
sp_run = fn.run(name='run_withcode',artifact_path="/User/artifacts",handler='save_property',params=params,project='Iguazio-Training',watch=False)

> 2020-08-05 13:15:10,445 [info] starting run run_withcode uid=39812776466f44b8b6b16b2a53b5a272  -> http://mlrun-api:8080
> 2020-08-05 13:15:10,600 [info] Job is running in the background, pod: run-withcode-lzhtr


project,uid,iter,start,state,name,labels,inputs,parameters,results,artifacts
Iguazio-Training,...53b5a272,0,Aug 05 13:15:10,running,run_withcode,v3io_user=adminkind=jobowner=admin,,source_file=/User/igztraining/mlrun/data/demo.csvtarget_file=/User/igztraining/mlrun/data/condos.csv,,


to track results use .show() or .logs() or in CLI: 
!mlrun get run 39812776466f44b8b6b16b2a53b5a272 --project Iguazio-Training , !mlrun logs 39812776466f44b8b6b16b2a53b5a272 --project Iguazio-Training
> 2020-08-05 13:15:10,619 [info] run executed, status=running


# Run function as part of a pipeline

In [29]:
import kfp
from kfp import dsl
from mlrun import run_pipeline

In [33]:
@dsl.pipeline(
    name = 'Property Management',
    description = 'Condo reviews'
)
def job_pipeline(
source_file :str = ''
) -> None:
    """Define our pipeline.
    :param p1: A model parameter.
    """
    save_properties = fn.as_step(handler='save_property',
                            params={"source_file" : source_file,
"target_file" : "/User/igztraining/mlrun/data/condos.csv" },outputs=['condo_properties'])
    filter_properties = fn.as_step(handler='filter_properties',
                                   params={"source_file" : source_file,
"target_file" : "/User/igztraining/mlrun/data/condos.csv" },
                            inputs={"condo_properties":save_properties.outputs["condo_properties"]})

In [34]:
kfp.compiler.Compiler().compile(job_pipeline, 'jobpipe.yaml')
artifact_path = 'v3io:///users/admin/kfp/{{workflow.uid}}/'
arguments = {"source_file" : "/User/igztraining/mlrun/data/demo.csv"}
run_id = run_pipeline(job_pipeline, arguments, experiment='property_management', artifact_path=artifact_path)

> 2020-08-05 13:41:33,810 [info] Pipeline run id=64ab066d-308a-480a-a9b7-edb10e05f990, check UI or DB for progress
